# `测试题`
## 0. 服务器是如何识访问来自浏览器还是非浏览器的？
## 请求中的header部分的user-Agent 字段 
## 1. 明明代码跟视频中的栗子一样，一运行却出错了，但在不修改代码的情况下再次尝试运行却又变好了，这是为什么呢？
## 服务器的响应质量时好时坏，网络可能丢包
## 2. Request 是由客户端发出还是由服务端发出？
## 客户端
## 3. 请问如何为一个 Request 对象动态的添加 headers？
## 要动态的添加headers对象，在Request对象创建后，使用Request的add_header方法进行添加
## 4. 简单来说，代理服务器是如何工作的？他有时为何不工作了？
## 代理服务器相当于一个中间人，客户端不能直接访问目标网页时，可以使用中间人访问转发。
## 这个中间人需要干活不要奖赏的话，罢工是正常的。
## 5. HTTP 有好几种方法（GET，POST，PUT，HEAD，DELETE，OPTIONS，CONNECT），请问你如何晓得 Python 是使用哪种方法访问服务器呢？
## get和post 吧
## 6. 上一节课后题中有涉及到登陆问题，辣么，你还记得服务器是通过什么来确定你是登陆还是没登陆的么？他会持续到什么时候呢？
## cookies
## 第一类cookies:即时过期的，称为“会话”cookies，当浏览器关闭时（python程序的请求结束时），会被自动清理
## 第二类cookies:有期限的，由浏览器进行存储，并在下一次请求该网站时自动附带（没过期或没被清理掉）

# `动动手：`
0. 编写一个爬虫，爬百度百科“网络爬虫”的词条（链接 -> http://baike.baidu.com/view/284853.htm ），将所有包含“view”的链接按下边格式打印出来：


In [16]:
import urllib.request
import re
from bs4 import BeautifulSoup


url = 'http://baike.baidu.com/view/284853.htm'
request = urllib.request.Request(url)
responce = urllib.request.urlopen(request)
html = responce.read()
soup = BeautifulSoup(html,'html.parser')
target_view = soup.find_all(href=re.compile("view"))
#print(soup)
print('-------------------------')
print(target_view)
print('-------------------------')
for each in target_view:
    print(each.text+ "->"+"http://baike.baidu.com"+each["href"])

-------------------------
[<a href="/wikicategory/view?categoryName=恐龙大全" target="_blank">恐龙百科</a>, <a href="/wikicategory/view?categoryName=多肉植物" target="_blank">多肉百科</a>]
-------------------------
恐龙百科->http://baike.baidu.com/wikicategory/view?categoryName=恐龙大全
多肉百科->http://baike.baidu.com/wikicategory/view?categoryName=多肉植物


## 1. 直接打印词条名和链接不算什么真本事儿，这题要求你的爬虫允许用户输入搜索的关键词。
## 然后爬虫进入每一个词条，然后检测该词条是否具有副标题（比如搜索“猪八戒”，副标题就是“（中国神话小说《西游记》的角色）”），如果有，请将副标题一并打印出来：

In [1]:
import urllib.request
import urllib.parse
import re
from bs4 import BeautifulSoup

while True:
    words = input("请输入关键词:")
    if words == '!q':
        break;
    words = urllib.parse.urlencode({"word":words})
    response = urllib.request.urlopen("http://baike.baidu.com/search/word?%s" % words)
    html = response.read()
    soup = BeautifulSoup(html, "html.parser")

    for each in soup.find_all(href=re.compile("view")):
        content = ''.join([each.text])
        url2 = ''.join(["http://baike.baidu.com", each["href"]])
        print(url2)
        #if 'wikicategory ' in url2:
        #    continue
        #response2 = urllib.request.urlopen(url2)
        #html2 = response2.read()
        #soup2 = BeautifulSoup(html2, "html.parser")
        #if soup2.h2:
        #    content = ''.join([content, soup2.h2.text])
        #content = ''.join([content, " -> ", url2])
        #print(content)




请输入关键词:兔子
http://baike.baidu.com/wikicategory/view?categoryName=恐龙大全
http://baike.baidu.com/wikicategory/view?categoryName=多肉植物
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/16846004#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/16134191#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/21500204#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/15818386#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/16248295#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/16815924#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/20272005#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/15874933#viewPageContent
http://baike.baidu.com/item/%E5%85%94%E5%AD%90/23397970#viewPageContent
http://baike.baidu.com/subview/71844/10028254.htm
请输入关键词:!q


# 2. 哗啦啦地丢一堆链接给用户可不是什么好的体验，我们应该先打印 10 个链接，然后问下用户“您还往下看吗？”
# 来，我给大家演示下：

# 然后为了增加用户体验，代码需要捕获未收录的词条，并提示：

In [3]:
import urllib.request
import urllib.parse
import re 
from bs4 import BeautifulSoup

def test_url(soup):
    result = soup.find(text=re.compile("百度百科尚未收录词条"))
    if result:
        print(result[0:-1]) # 百度这个碧池在最后加了个“符号，给它去掉
        return False
    else:
        return True

def summary(soup):
    word = soup.h1.text
    # 如果存在副标题，一起打印
    if soup.h2:
        word += soup.h2.text
    # 打印标题
    print(word)
    # 打印简介
    if soup.find(class_="lemma-summary"):
        print(soup.find(class_="lemma-summary").text)   

def get_urls(soup):
    for each in soup.find_all(href=re.compile("view")):
        content = ''.join([each.text])
        url2 = ''.join(["http://baike.baidu.com", each["href"]])
        response2 = urllib.request.urlopen(url2)
        html2 = response2.read()
        soup2 = BeautifulSoup(html2, "html.parser")
        if soup2.h2:
            content = ''.join([content, soup2.h2.text])
        content = ''.join([content, " -> ", url2])
        yield content
        

def main():
    word = input("请输入关键词：")
    keyword = urllib.parse.urlencode({"word":word})
    response = urllib.request.urlopen("http://baike.baidu.com/search/word?%s" % keyword)
    html = response.read()
    soup = BeautifulSoup(html, "html.parser")

    if test_url(soup):
        summary(soup)
        
        print("下边打印相关链接：")
        each = get_urls(soup)
        while True:
            try:
                for i in range(10):
                    print(next(each))
            except StopIteration:
                break
            
            command = input("输入任意字符将继续打印，q退出程序：")
            if command == 'q':
                break
            else:
                continue
    
if __name__ == "__main__":
    main()

请输入关键词：小说
小说（文学体裁）

小说，以刻画人物形象为中心，通过完整的故事情节和环境描写来反映社会生活的文学体裁。人物、情节、环境是小说的三要素。情节一般包括开端、发展、高潮、结局四部分，有的包括序幕、尾声。环境包括自然环境和社会环境。        小说按照篇幅及容量可分为长篇、中篇、短篇和微型小说（小小说）。按照表现的内容可分为神话、仙侠、武侠、古传、当代等。按照体制可分为章回体小说、日记体小说、书信体小说、自传体小说。按照语言形式可分为文言小说和白话小说。小说与诗歌、散文、戏剧，并称“四大文学体裁”。小说刻画人物的方法：心理描写、动作描写、语言描写、外貌描写、神态描写，同时，小说是一种写作方法。

下边打印相关链接：


UnicodeEncodeError: 'ascii' codec can't encode characters in position 36-39: ordinal not in range(128)